In [0]:
silver = spark.read.format("delta").load("/Volumes/workspace/ecommerce/delta/silver/events")

In [0]:
silver.show()

In [0]:
%sql
create table if not exists products
using delta
as select * from delta.`/Volumes/workspace/ecommerce/delta/silver/events`;

### user-level conversion rate

In [0]:
%sql

WITH viewed AS (
    SELECT DISTINCT user_id, product_category
    FROM products
    WHERE event_type = 'view'
),
purchased AS (
    SELECT DISTINCT user_id, product_category
    FROM products
    WHERE event_type = 'purchase'
)
SELECT
    v.product_category,
    ROUND(
        COUNT(DISTINCT p.user_id) * 100.0 / NULLIF(COUNT(DISTINCT v.user_id), 0),
        2
    ) AS conversion_rate
FROM viewed v
LEFT JOIN purchased p
    ON v.user_id = p.user_id
   AND v.product_category = p.product_category
GROUP BY v.product_category
ORDER BY conversion_rate DESC limit 10;


### Top 10 smartphone brands by highest sales quantity

In [0]:
%sql
SELECT brand, count(*) AS quantities_sale FROM products
WHERE event_type == 'purchase' AND product_category = 'smartphone'
GROUP BY brand
ORDER BY quantities_sale DESC
LIMIT 10;

### Revenue with 7-day moving average

In [0]:
%sql
with daily as (select event_date, round(sum(price)) as revenue from products
where event_type = 'purchase'
group by event_date)

select event_date, revenue,
       round(avg(revenue) over(order by event_date rows between 6 preceding and current row)) as avg_7d
from daily
 

## event-level conversion rate

In [0]:
%sql
WITH view_table AS (
    SELECT 
        user_id, 
        product_category, 
        COUNT(*) AS viewed
    FROM products
    WHERE event_type = 'view'
    GROUP BY user_id, product_category
),
purchase_table AS (
    SELECT 
        user_id, 
        product_category, 
        COUNT(*) AS purchased
    FROM products
    WHERE event_type = 'purchase'
    GROUP BY user_id, product_category
)
SELECT 
    v.product_category,
    ROUND(
        COALESCE(SUM(p.purchased), 0) * 100.0 / NULLIF(SUM(v.viewed), 0),
        2
    ) AS conversion_rate
FROM view_table v
LEFT JOIN purchase_table p
    ON v.user_id = p.user_id
   AND v.product_category = p.product_category
GROUP BY v.product_category
ORDER BY conversion_rate DESC;


In [0]:
%sql
select 
   case when cnt >= 10 then "VIP"
        when cnt >= 5 then "Loyal"
        else "Regular" end as tier,
        count(*) as customers,
        avg(total_spent) avg_spend
from (
select user_id, count(*) cnt, sum(price) total_spent
from products where event_type = 'purchase' group by user_id)
group by tier